# Let's Build a Multi Agent Example Live!

In [1]:
import nest_asyncio
nest_asyncio.apply()

from agents import Agent, Runner
from agents.tool import WebSearchTool
from agents import function_tool

@function_tool
def write_to_file(path: str, content: str):
    with open(path, "w") as f:
        f.write(content)
    return f"File '{path}' written successfully."

@function_tool
def read_file(path: str):
    with open(path, "r") as f:
        return f.read()

In [11]:
# --- Modified agent definitions to support returning outputs to triage agent ---

research_agent = Agent(
    name="ResearchAgent",
    instructions="""
    You are a research assistant.
    You are given a question and you need to find the answer by searching the web.
    Return your findings as a string to the triage agent.
    """,
    model="gpt-5-mini",
    tools=[WebSearchTool()],
    # Ensure outputs are returned to the caller (triage agent)
)

file_management_agent = Agent(
    name="FileManagementAgent",
    instructions="""
    You are a file management assistant.
    You are given a file path and you need to manage the file.
    Return the result of your operation to the triage agent.
    """,
    model="gpt-5-mini",
    tools=[read_file, write_to_file],
)


In [12]:
# According to the OpenAI Agents SDK, to ensure that sub-agents (handoffs) return their output to the triage agent,
# you simply define the handoffs list on the triage agent. The SDK automatically returns the sub-agent's output
# to the calling agent (here, the triage agent), which can then process or chain outputs as needed.
# There is no need for an `allow_multi_handoff` flag; just ensure your instructions are clear.

triage_agent = Agent(
    name="TriageAgent",
    instructions="""
    You are a triage assistant.
    You are given a user request and you need to determine which agent(s) to hand off the request to.
    You may need to use multiple agents in sequence, passing outputs from one agent as inputs to another.
    The agents you can hand off to are:
    - ResearchAgent (for web search tasks)
    - FileManagementAgent (for file management tasks like write or read)
    If a task requires both research and file management (e.g., research then write to file), first call ResearchAgent, then pass its output to FileManagementAgent.
    Always return the final result to the user.
    """,
    model="gpt-5-mini",
    handoffs=[research_agent, file_management_agent],
)

In [13]:
research_agent.handoffs = [triage_agent]

file_management_agent.handoffs = [triage_agent]

In [15]:
# --- Example 1: Simple research task ---
result = Runner.run_sync(triage_agent, "What are the top AI tools that came out in August 2025?  Write summary to file report.md")
print("Result 1:", result.final_output)

Result 1: Handoff complete.

I have transferred this request to the ResearchAgent (to gather the top AI tools released in August 2025) and to the FileManagementAgent (to write the summary to the specified file). Target output: a concise summary saved to report.md.

Details for the agents:
- Task for ResearchAgent: Identify the top AI tools released in August 2025 (with source links), and produce a 300–600 word summary highlighting each tool’s name, developer, core capabilities, notable use cases, pricing/licensing (if known), and why it stands out.
- Task for FileManagementAgent: Write the ResearchAgent’s summary exactly into report.md in the working directory and confirm write success (include file path and byte/line count).

File to create/update: report.md

Status: Transferred to ResearchAgent and FileManagementAgent. Please proceed and report back when the file has been written.


In [16]:
from pydantic import BaseModel, Field
from typing import List

class TaskPlan(BaseModel):
    tasks: List[str] = Field(description="A list of tasks to complete created from the input prompt.")

planner_agent = Agent(
    name="PlannerAgent",
    instructions="""
    You are a planner assistant.
    You are given a user request and you need to determine which agent(s) to hand off the request to.
    You may need to use multiple agents in sequence, passing outputs from one agent as inputs to another.
    The agents you can hand off to are:
    - ResearchAgent (for web search tasks)
    - FileManagementAgent (for file management tasks like write or read)
    You need to create a list of tasks to complete the user request.
    Return the list of tasks to the triage agent.
    """,
    model="gpt-5-mini",
    output_type=TaskPlan,
)


result = Runner.run_sync(planner_agent, "What are the top AI tools that came out in August 2025?  Write summary to file report.md")
print("Result:", result.final_output)

Result: tasks=['Use ResearchAgent to search the web for AI tools and products first released or announced in August 2025. Collect candidate items from reputable sources (press releases, major tech news outlets, company blogs, product pages, GitHub releases).', 'From the collected candidates, shortlist the top ~8–12 AI tools released in August 2025, using criteria: novelty/technical advance, breadth of impact or adoption, coverage in reputable media, and clear August 2025 release/announcement date.', 'For each shortlisted tool, gather structured metadata: official name, developer/company, exact release/announcement date in August 2025, one-paragraph description, 3–5 key features or capabilities, target users/use cases, pricing or availability details, and 1–2 source links.', 'Compose a concise report (report.md) that includes: a brief intro explaining scope and selection criteria, a short entry for each top tool with the collected metadata and a 2–3 sentence summary of why it matters, a

In [18]:
context_history = []
for task in result.final_output.tasks:
    # Combine all previous results as context for the current task
    context_str = "\n".join(context_history)
    if context_str:
        task_with_context = f"Context from previous tasks:\n{context_str}\n\nCurrent task:\n{task}"
    else:
        task_with_context = f"Current task:\n{task}"
    task_result = Runner.run_sync(triage_agent, task_with_context)
    print("Result:", task_result.final_output)
    context_history.append(task_result.final_output)

Result: ResearchAgent — please search the web for AI tools and products that were first released or announced in August 2025. Collect candidate items from reputable sources (press releases, major tech news outlets, company blogs, product pages, GitHub releases). Return your findings as a single string listing each candidate item with a 1–2 sentence summary and at least one source citation (URL or source name).
Result: Handoff complete — I’ve transferred this to the ResearchAgent to search the web for AI tools/products announced or released in August 2025 and to return candidate items (each with a 1–2 sentence summary and source citations) as a single string. Once that research output is returned, I can shortlist the top ~8–12 items per your criteria. If you want any specific sources prioritized (e.g., company blogs, TechCrunch, The Verge), tell me now.
Result: I've transferred this to the ResearchAgent to collect structured metadata for each shortlisted tool. Please provide the shortli

In [ ]:
import asyncio
import uuid

from openai.types.responses import ResponseContentPartDoneEvent, ResponseTextDeltaEvent

from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace

"""
This example shows the handoffs/routing pattern. The triage agent receives the first message, and
then hands off to the appropriate agent based on the language of the request. Responses are
streamed to the user.
"""

french_agent = Agent(
    name="french_agent",
    instructions="You only speak French",
)

spanish_agent = Agent(
    name="spanish_agent",
    instructions="You only speak Spanish",
)

english_agent = Agent(
    name="english_agent",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="triage_agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[french_agent, spanish_agent, english_agent],
)


async def main():
    # We'll create an ID for this conversation, so we can link each trace
    conversation_id = str(uuid.uuid4().hex[:16])

    msg = input("Hi! We speak French, Spanish and English. How can I help? ")
    agent = triage_agent
    inputs: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

    while True:
        # Each conversation turn is a single trace. Normally, each input from the user would be an
        # API request to your app, and you can wrap the request in a trace()
        with trace("Routing example", group_id=conversation_id):
            result = Runner.run_streamed(
                agent,
                input=inputs,
            )
            async for event in result.stream_events():
                if not isinstance(event, RawResponsesStreamEvent):
                    continue
                data = event.data
                if isinstance(data, ResponseTextDeltaEvent):
                    print(data.delta, end="", flush=True)
                elif isinstance(data, ResponseContentPartDoneEvent):
                    print("\n")

        inputs = result.to_input_list()
        print("\n")

        user_msg = input("Enter a message: ")
        inputs.append({"content": user_msg, "role": "user"})
        agent = result.current_agent


if __name__ == "__main__":
    asyncio.run(main())